# Setup Code (Run first)

In [10]:
%%capture

# Capture the output but do not do anything with it
# Just to avoid cluttering the output
%pip install --upgrade git+https: // github.com/terrierteam/pyterrier_doc2query.git
%pip install python-terrier == 0.9.2
%pip install semanticscholar
%pip install gensim
%pip install seaborn
%pip install KrovetzStemmer
%pip install nltk
%pip install langdetect


# Load all packages and initialize pyTerrier
import nltk
import numpy as np
import pandas as pd
import pyterrier as pt
import os
from gensim.models import Word2Vec
from semanticscholar import SemanticScholar
from krovetzstemmer import Stemmer
from nltk.tokenize import RegexpTokenizer
from langdetect import detect

krovetz_stemmer = Stemmer()
regex_tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')

if not pt.started():
    pt.init()

index_count = 0
# Helper function to intialize multiple indices
# Prepares the index path and avoid errors with already existing indices
def prepare_index_path(indexName):
    global index_count
    index_count = index_count + 1
    index_path = 'indices/' + indexName + str(index_count)

    if os.path.exists(index_path) & os.path.isdir(index_path):
        files = os.listdir(index_path)
        for file in files:
            file_name = index_path + '/' + file
            os.remove(file_name)
        os.rmdir(index_path)
    elif os.path.exists(index_path) & (not os.path.isdir(index_path)):
        os.rmove(index_path)

    return os.path.abspath(index_path)

def build_index(indexName, dataset):
    index_path = prepare_index_path(indexName)
    indexer = pt.IterDictIndexer(
        index_path, overwrite=True, blocks=True)
    indexer.setProperty(
        "stopwords.filename", "F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\en.txt")
    index_created = indexer.index(dataset.get_corpus_iter(),
                            fields=['title', 'doi', 'abstract'],
                            meta=('docno',))
    return index_created

# Vorgehen

## Allgemein

TREC-Covid-Queries verwenden und erweitern um weitere Anfrageterme, um die Retrieval-Performance zu verbessern (Query Expansion). 

Für die QE müssen Termkandidaten ("set of C" c_1, c_2, c_3, ...) bestimmt werden, die anschließend gerankt werden.
D.h. ihr sendet zunächst die Standard-Query ab und später nochmal für die finale Evaluierung, die Query mit Termerweiterungen.

## QE 
Wenn wenig Zeit:
Nur die globale Variante evaluieren, wobei ja der "Claim for Fame" im Paper ist, dass die lokale Methode bessere Ergebnisse liefert.
### LOKALE QE 
Word2Vec-Embeddings auf TREC-Covid trainieren (https://radimrehurek.com/gensim/)


In [8]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [6]:
# Use SMART Stopwordlist to remove stopwords
def remove_stopwords(text):
    with open('en.txt') as f:
        lines = [line.rstrip('\n') for line in f]
    stop_words = set(lines)
    return " ".join([word for word in text.split() if word.lower() not in stop_words])
# Preprocess the given text with SMART Stopword list, regex-Tokenizer and lowercase conversion
def preprocess(text):
    sentences = nltk.sent_tokenize(text)
    tokens_list = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = remove_stopwords(sentence)

    tokens = regex_tokenizer.tokenize(sentence)

    result = []
    for token in tokens:
        result.append(krovetz_stemmer.stem(token))
    return result

In [9]:
docs = pd.DataFrame(dataset.get_corpus_iter())
abstracts = docs["abstract"]
my_df = pd.DataFrame(abstracts)

print(abstracts[0])

cord19/trec-covid documents: 100%|██████████| 192509/192509 [00:01<00:00, 159974.80it/s]


OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were t

In [ ]:
languages = []
for i, doc in docs.iterrows():
    text = doc['abstract']

    if text is pd.NA or text == "":
        text = doc['title']
    try:                                                          
        lang = detect(text)                         
    except Exception as e:                                                
        lang='unknown'
    languages.append(lang)

languages = np.asarray(languages)
v, c = np.unique(languages, return_counts=True)

In [ ]:
model = Word2Vec(sentences=abstracts,window=10,sg=1,seed=1)
model.save("word2vec_abstracts.model")

In [44]:
model.wv.most_similar('c', topn=1)

[('s', 0.9356473684310913)]

In [14]:
from sklearn.decomposition import PCA
from matplotlib import pyplot
# fit a 2d PCA model to the vectors
X = model[model.wv.key_to_index]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.key_to_index)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

TypeError: 'Word2Vec' object is not subscriptable

### GLOBALE QE
Standardvariante wie z.B. auf Basis von Wikipedia.

# 1. Standard-Query an den Index für erstes Ranking
Aus dem Title des Topics, quasi wie in der Standard-Pipeline in Pyterrier.
Paper Inverse Document Frequency model (InL2)
Zuerst BM25.

In [ ]:
# Create index only to retrieve the tokens from it
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
trec_covid_index_ref = build_index('trecCovid',dataset)
trec_covid_index = pt.IndexFactory.of(trec_covid_index_ref)

# 2. Erstellung der Termkandidaten
## 2.1 Top-3 Dokumente
Alle Terme (ausschließlich der Stoppworte).

## 2.2 Terme der Referenzen
Alle Terme aus den Referenzen der 3 Dokumente (Terme der Titel).

## 2.3 Erweiterung mit Co-Autoren
Über die Koautoren können noch weitere Dokumente bzw. die entsprechenden Terme hinzugefügt werden. Aus Zeitgründen könnte man vielleicht auch diesen Schritt weglassen, wenn es zu viel Aufwand ist über Koautoren weitere Dokumente zu finden. Bei der Implementierung scheint ihr ja aber schon recht weit zu sein. Die "relevanten Paper der Autoren" sind einfach die Top-k Dokumente oder möglicherweise alle zusätzlichen Papers, die über die Koautorenschaft gefunden werden, gemeint. Daher ergibt sich auf der Name PSEUDO-Relevanz-Feedback, da einfach angenommen wird, dass die Top-Treffer alle relevant sind.

# 2.4 Ranking aller Terms
Paper: Bo1
Top-k Terme auswählen (k selbst wählen).
Ergebnis: Potentielle Kandidaten für eine Termwerweiterung

# 2.5 Word2Vec-Modell mit Terms nachtrainieren
Über die Kosinusähnlichkeit der Embeddings der ursprünglichen Anfrageterme bestimmt ihr nun weitere Terme aus dem Word2Vec-Modell. Diese Termkandidaten werden dann wie in 2.1.4 über Bo1 o.ä. gerankt.

In [ ]:
mv_model = Word2Vec.load("data/word2vec.model")

queries = preprocess_queries(queries, augmented=True, mv_model=mv_model)
display(queries.head(5))

pq_title = queries[['query_id', 'title']]
pq_title = pq_title.rename(columns={'query_id':'qid', 'title':'query'})
display(pq_title.head(5))

pq_desc = queries[['query_id', 'description']]
pq_desc = pq_desc.rename(columns={'query_id':'qid', 'description':'query'})
display(pq_desc.head(5))

pq_narr = queries[['query_id', 'narrative']]
pq_narr = pq_narr.rename(columns={'query_id':'qid', 'narrative':'query'})
display(pq_narr.head(5))

# 2.6 Zwei Sets zusammenfügen und Top-k für QE der ursprünglichen Querry nutzen
Diese jeweils für die 50 Topics absenden.

Ergebnis: Die finalen Rankings, die dann ausgewertet werden können.